In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import sys
from datetime import date, timedelta

In [5]:
spark = SparkSession \
    .builder \
    .appName("Brewery") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/29 21:24:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
df = spark.read.json("bronze/data.json",multiLine=True)

In [52]:
df.show()

+--------------------+--------------+--------------+------------+--------------+-------------+--------------------+-----------------+------------------+--------------------+-------------+-------------------+-------------+-------------------+--------------------+--------------------+
|      name_address_1|name_address_2|name_address_3|brewery_type|     name_city| name_country|          id_brewery| latitude_brewery| longitude_brewery|        name_brewery|phone_brewery|postal_code_brewery|   name_state|name_state_province|         name_street|         website_url|
+--------------------+--------------+--------------+------------+--------------+-------------+--------------------+-----------------+------------------+--------------------+-------------+-------------------+-------------+-------------------+--------------------+--------------------+
|      1716 Topeka St|          NULL|          NULL|       micro|        Norman|United States|5128df48-79fc-4f0...|      35.25738891|      -97.46818

In [15]:
df = df.withColumnRenamed("address_1", "name_address_1") \
        .withColumnRenamed("address_2", "name_address_2") \
        .withColumnRenamed("address_3", "name_address_3") \
        .withColumnRenamed("street", "name_street") \
        .withColumnRenamed("city", "name_city") \
        .withColumnRenamed("country", "name_country") \
        .withColumnRenamed("name", "name_brewery") \
        .withColumnRenamed("id", "id_brewery") \
        .withColumnRenamed("state", "name_state") \
        .withColumnRenamed("state_province", "name_state_province") \
        .withColumnRenamed("phone", "phone_brewery") \
        .withColumnRenamed("latitude", "latitude_brewery") \
        .withColumnRenamed("longitude", "longitude_brewery") \
        .withColumnRenamed("postal_code", "postal_code_brewery")

In [62]:
if df.filter(df.name_state.isNull()).count() >= 1 or df.filter(df.brewery_type.isNull()).count() >= 1:
    print("has null values")
    sys.exit()
else:
    print("has no null values")

has no null values


In [76]:
today = date.today()

In [77]:
df = df.withColumn("dt_load", lit(today))

In [82]:
df.write.format("parquet").mode("append")\
        .partitionBy(["name_state"])\
        .option("mergeSchema", "true").save(f'silver/day{today}')